In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import time
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

from pprint import pprint

import src
from src.reload import deep_reload

In [2]:
directory = 'Outputs/Experiment/'

files = os.listdir(directory)

In [21]:
risk_attitudes = ['Aggressive', 'Neutral', 'Cautious']
risk_attitudes = [.25, .5, .75]
risk_attitudes_ranges = [(0, .5), (0, 1), (.5, 1)]
ranges = [200, 300, 400]
reliabilities = [.50, .75, 1]
sngs = ['Tesla', 'Other', 'Combined']

levels = src.utilities.full_factorial([3, 3, 3, 3])

In [27]:
deep_reload(src)

rta = []
ra = []
rng = []
rel = []
sng = []

for idx, file in enumerate(files):
    # print(idx)

    row = levels[idx]

    try:
        costs, values, paths = pkl.load(open(f'Outputs/Experiment/case_{idx}.pkl', 'rb'))
    
        rta_c = src.routing.road_trip_accessibility(
            values,
            expectation = lambda x: src.routing.super_quantile(
                x, risk_attitudes_ranges[row[0]])
        ) / 3600
    
        rta.append(rta_c)
        ra.append(risk_attitudes[row[0]])
        rng.append(ranges[row[1]])
        rel.append(reliabilities[row[2]])
        sng.append(sngs[row[3]])
        
    except:
        print(f'python run_experiment.py -i {idx} &')
        print(idx, sum([type(v) is str for v in values.values()]))

    

In [28]:
rng_n = [(r - 200) / 200 for r in rng]
rel_n = [(r - .5) / .5 for r in rel]
ra_n = [(r - .25) / .5 for r in ra]

In [29]:
data = {'Attitude': ra_n, 'Range': rng_n, 'Reliability': rel_n, 'Network': sng, 'RTA': rta}

df = pd.DataFrame(
    data = data,
)

In [10]:
df_sel = df[df['Network'] == 'Tesla']
# df_sel = df

model = smf.ols(
    'RTA ~ C(Attitude) * Range * Reliability * C(Network)', data = df_sel
).fit()

In [11]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    RTA   R-squared:                       0.903
Model:                            OLS   Adj. R-squared:                  0.832
Method:                 Least Squares   F-statistic:                     12.71
Date:                Wed, 15 May 2024   Prob (F-statistic):           1.09e-05
Time:                        00:55:10   Log-Likelihood:                -3.2297
No. Observations:                  27   AIC:                             30.46
Df Residuals:                      15   BIC:                             46.01
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                     6.4661      0.305     21.207      0.000       5.816       7.116
C(Attitude)[T.Cautious]                       2.4845      0.431      5.762      0.000       1.565       3.404
C(Attitude)[T.Neutral]                        1.2826      0.431      2.975      0.009       0.364       2.202
Range                                        -0.9326      0.472     -1.974      0.067      -1.939       0.074
C(Attitude)[T.Cautious]:Range                -2.5071      0.668     -3.753      0.002      -3.931      -1.083
C(Attitude)[T.Neutral]:Range                 -1.2918      0.668     -1.934      0.072      -2.716       0.132
Reliability                                  -0.2855      0.472     -0.604      0.555      -1.292       0.721
C(Attitude)[T.Cautious]:Reliability          -2.2579      0.668     -3.380      0.004      -3.682      -0.834
C(Attitude)[T.Neutral]:Reliability           -1.1634      0.668     -1.742      0.102      -2.587       0.260
Range:Reliability                             0.3119      0.732      0.426      0.676      -1.248       1.872
C(Attitude)[T.Cautious]:Range:Reliability     2.2188      1.035      2.144      0.049       0.013       4.425
C(Attitude)[T.Neutral]:Range:Reliability      1.1403      1.035      1.102      0.288      -1.065       3.346
==============================================================================
Omnibus:                        1.619   Durbin-Watson:                   1.089
Prob(Omnibus):                  0.445   Jarque-Bera (JB):                0.795
Skew:                          -0.409   Prob(JB):                        0.672
Kurtosis:                       3.194   Cond. No.                         37.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
df_sel = df[df['Network'] == 'Other']

model = smf.ols(
    'RTA ~ C(Attitude) * Range * Reliability * C(Network)', data = df_sel
).fit()

In [13]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    RTA   R-squared:                       0.922
Model:                            OLS   Adj. R-squared:                  0.865
Method:                 Least Squares   F-statistic:                     16.20
Date:                Wed, 15 May 2024   Prob (F-statistic):           2.25e-06
Time:                        00:55:10   Log-Likelihood:                -10.106
No. Observations:                  27   AIC:                             44.21
Df Residuals:                      15   BIC:                             59.76
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                     6.8630      0.393     17.448      0.000       6.025       7.701
C(Attitude)[T.Cautious]                       4.0398      0.556      7.262      0.000       2.854       5.225
C(Attitude)[T.Neutral]                        1.8702      0.556      3.362      0.004       0.685       3.056
Range                                        -0.9900      0.609     -1.625      0.125      -2.289       0.309
C(Attitude)[T.Cautious]:Range                -3.9683      0.862     -4.605      0.000      -5.805      -2.131
C(Attitude)[T.Neutral]:Range                 -1.8233      0.862     -2.116      0.052      -3.660       0.014
Reliability                                  -0.0060      0.609     -0.010      0.992      -1.305       1.293
C(Attitude)[T.Cautious]:Reliability          -3.1349      0.862     -3.638      0.002      -4.972      -1.298
C(Attitude)[T.Neutral]:Reliability           -1.3674      0.862     -1.587      0.133      -3.204       0.469
Range:Reliability                             0.0529      0.944      0.056      0.956      -1.959       2.065
C(Attitude)[T.Cautious]:Range:Reliability     3.1207      1.335      2.337      0.034       0.275       5.966
C(Attitude)[T.Neutral]:Range:Reliability      1.3471      1.335      1.009      0.329      -1.499       4.193
==============================================================================
Omnibus:                        4.445   Durbin-Watson:                   1.155
Prob(Omnibus):                  0.108   Jarque-Bera (JB):                2.774
Skew:                          -0.715   Prob(JB):                        0.250
Kurtosis:                       3.648   Cond. No.                         37.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
df_sel = df[df['Network'] == 'Combined']

model = smf.ols(
    'RTA ~ C(Attitude) * Range * Reliability * C(Network)', data = df_sel
).fit()

In [15]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    RTA   R-squared:                       0.918
Model:                            OLS   Adj. R-squared:                  0.858
Method:                 Least Squares   F-statistic:                     15.31
Date:                Wed, 15 May 2024   Prob (F-statistic):           3.25e-06
Time:                        00:55:10   Log-Likelihood:                -1.8756
No. Observations:                  27   AIC:                             27.75
Df Residuals:                      15   BIC:                             43.30
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                     6.6650      0.290     22.983      0.000       6.047       7.283
C(Attitude)[T.Cautious]                       2.6333      0.410      6.421      0.000       1.759       3.507
C(Attitude)[T.Neutral]                        1.3569      0.410      3.309      0.005       0.483       2.231
Range                                        -0.8906      0.449     -1.982      0.066      -1.848       0.067
C(Attitude)[T.Cautious]:Range                -2.5704      0.635     -4.046      0.001      -3.925      -1.216
C(Attitude)[T.Neutral]:Range                 -1.3237      0.635     -2.083      0.055      -2.678       0.031
Reliability                                  -0.0743      0.449     -0.165      0.871      -1.032       0.883
C(Attitude)[T.Cautious]:Reliability          -2.0281      0.635     -3.192      0.006      -3.382      -0.674
C(Attitude)[T.Neutral]:Reliability           -1.0468      0.635     -1.648      0.120      -2.401       0.307
Range:Reliability                             0.1185      0.696      0.170      0.867      -1.365       1.602
C(Attitude)[T.Cautious]:Range:Reliability     1.9973      0.984      2.029      0.061      -0.101       4.095
C(Attitude)[T.Neutral]:Range:Reliability      1.0289      0.984      1.045      0.312      -1.069       3.127
==============================================================================
Omnibus:                        3.146   Durbin-Watson:                   1.092
Prob(Omnibus):                  0.207   Jarque-Bera (JB):                1.902
Skew:                          -0.629   Prob(JB):                        0.386
Kurtosis:                       3.327   Cond. No.                         37.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [30]:
df_sel = df

model = smf.ols(
    'RTA ~ Attitude * Range * Reliability * C(Network)', data = df_sel
).fit()

In [31]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    RTA   R-squared:                       0.923
Model:                            OLS   Adj. R-squared:                  0.892
Method:                 Least Squares   F-statistic:                     29.68
Date:                Wed, 15 May 2024   Prob (F-statistic):           8.80e-24
Time:                        07:36:33   Log-Likelihood:                -16.868
No. Observations:                  81   AIC:                             81.74
Df Residuals:                      57   BIC:                             139.2
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
==================================================================================================================
                                                     coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------
Intercept                                          6.6784      0.270     24.714      0.000       6.137       7.220
C(Network)[T.Other]                                0.1347      0.382      0.352      0.726      -0.631       0.900
C(Network)[T.Tesla]                               -0.1989      0.382     -0.520      0.605      -0.964       0.566
Attitude                                           2.6333      0.419      6.290      0.000       1.795       3.472
Attitude:C(Network)[T.Other]                       1.4065      0.592      2.376      0.021       0.221       2.592
Attitude:C(Network)[T.Tesla]                      -0.1488      0.592     -0.251      0.802      -1.334       1.037
Range                                             -0.9034      0.419     -2.158      0.035      -1.742      -0.065
Range:C(Network)[T.Other]                         -0.0330      0.592     -0.056      0.956      -1.219       1.153
Range:C(Network)[T.Tesla]                         -0.0419      0.592     -0.071      0.944      -1.227       1.144
Attitude:Range                                    -2.5704      0.649     -3.963      0.000      -3.869      -1.272
Attitude:Range:C(Network)[T.Other]                -1.3979      0.917     -1.524      0.133      -3.235       0.439
Attitude:Range:C(Network)[T.Tesla]                 0.0633      0.917      0.069      0.945      -1.773       1.900
Reliability                                       -0.0852      0.419     -0.203      0.840      -0.923       0.753
Reliability:C(Network)[T.Other]                    0.1458      0.592      0.246      0.806      -1.040       1.331
Reliability:C(Network)[T.Tesla]                   -0.2119      0.592     -0.358      0.722      -1.397       0.974
Attitude:Reliability                              -2.0281      0.649     -3.127      0.003      -3.327      -0.729
Attitude:Reliability:C(Network)[T.Other]          -1.1068      0.917     -1.207      0.233      -2.943       0.730
Attitude:Reliability:C(Network)[T.Tesla]          -0.2298      0.917     -0.250      0.803      -2.066       1.607
Range:Reliability                                  0.1286      0.649      0.198      0.844      -1.170       1.427
Range:Reliability:C(Network)[T.Other]             -0.1468      0.917     -0.160      0.873      -1.983       1.690
Range:Reliability:C(Network)[T.Tesla]              0.1936      0.917      0.211      0.834      -1.643       2.030
Attitude:Range:Reliability                         1.9973      1.005      1.988      0.052      -0.015       4.009
Attitude:Range:Reliability:C(Network)[T.Other]     1.1234      1.421      0.791      0.432      -1.722       3.969
Attitude:Range:Reliability:C(Network)[T.Tesla]     0.2215      1.421      0.156      0.877   

In [34]:
deep_reload(src)

label_substitutions={
    'C(Network)[T.Other]': 'Non-Tesla',
}

out_string=src.analysis.PrintLaTeXTabular(
    model, alpha=.05,
    label_substitutions = label_substitutions
)

print(out_string)

\hline {\small Intercept } & 6.678 & 24.714 & 0.000 \\
\hline {\small Range } & -0.903 & -2.158 & 0.035 \\
\hline {\small Attitude } & 2.633 & 6.290 & 0.000 \\
\hline {\small Attitude:Range } & -2.570 & -3.963 & 0.000 \\
\hline {\small Attitude:Non-Tesla } & 1.406 & 2.376 & 0.021 \\
\hline {\small Attitude:Reliability } & -2.028 & -3.127 & 0.003 \\



In [106]:
risk_attitudes = ['Aggressive', 'Neutral', 'Cautious']
risk_attitudes = [.25, .5, .75]
risk_attitudes_ranges = [(0, .5), (0, 1), (.5, 1)]
ranges = [200, 300, 400]
reliabilities = [.50, .75, 1]
sngs = ['Tesla', 'Other', 'Combined']

levels = src.utilities.full_factorial([3, 3, 3, 3])

In [107]:
deep_reload(src)

rta = []
ra = []
rng = []
rel = []
sng = []

for idx, file in enumerate(files):
    # print(idx)

    row = levels[idx]

    try:
        costs, values, paths = pkl.load(open(f'Outputs/Experiment/case_{idx}.pkl', 'rb'))
    
        rta_c = src.routing.road_trip_accessibility(
            values,
            # expectation = lambda x: src.routing.super_quantile(
            #     x, risk_attitudes_ranges[row[0]])
        ) / 3600
    
        rta.append(rta_c)
        ra.append(risk_attitudes[row[0]])
        rng.append(ranges[row[1]])
        rel.append(reliabilities[row[2]])
        sng.append(sngs[row[3]])
        
    except:
        print(f'python run_experiment.py -i {idx} &')
        print(idx, sum([type(v) is str for v in values.values()]))

    

In [108]:
rng_n = [(r - 200) / 200 for r in rng]
rel_n = [(r - .5) / .5 for r in rel]
ra_n = [(r - .25) / .5 for r in ra]

In [109]:
data = {'Attitude': ra_n, 'Range': rng_n, 'Reliability': rel_n, 'Network': sng, 'RTA': rta}

df1 = pd.DataFrame(
    data = data,
)

In [110]:
df_sel = df1

model = smf.ols(
    'RTA ~ Attitude * Range * Reliability * C(Network)', data = df_sel
).fit()

In [111]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    RTA   R-squared:                       0.923
Model:                            OLS   Adj. R-squared:                  0.892
Method:                 Least Squares   F-statistic:                     29.68
Date:                Wed, 15 May 2024   Prob (F-statistic):           8.80e-24
Time:                        09:18:33   Log-Likelihood:                -16.868
No. Observations:                  81   AIC:                             81.74
Df Residuals:                      57   BIC:                             139.2
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
==================================================================================================================
                                                     coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------
Intercept                                          6.6784      0.270     24.714      0.000       6.137       7.220
C(Network)[T.Other]                                0.1347      0.382      0.352      0.726      -0.631       0.900
C(Network)[T.Tesla]                               -0.1989      0.382     -0.520      0.605      -0.964       0.566
Attitude                                           2.6333      0.419      6.290      0.000       1.795       3.472
Attitude:C(Network)[T.Other]                       1.4065      0.592      2.376      0.021       0.221       2.592
Attitude:C(Network)[T.Tesla]                      -0.1488      0.592     -0.251      0.802      -1.334       1.037
Range                                             -0.9034      0.419     -2.158      0.035      -1.742      -0.065
Range:C(Network)[T.Other]                         -0.0330      0.592     -0.056      0.956      -1.219       1.153
Range:C(Network)[T.Tesla]                         -0.0419      0.592     -0.071      0.944      -1.227       1.144
Attitude:Range                                    -2.5704      0.649     -3.963      0.000      -3.869      -1.272
Attitude:Range:C(Network)[T.Other]                -1.3979      0.917     -1.524      0.133      -3.235       0.439
Attitude:Range:C(Network)[T.Tesla]                 0.0633      0.917      0.069      0.945      -1.773       1.900
Reliability                                       -0.0852      0.419     -0.203      0.840      -0.923       0.753
Reliability:C(Network)[T.Other]                    0.1458      0.592      0.246      0.806      -1.040       1.331
Reliability:C(Network)[T.Tesla]                   -0.2119      0.592     -0.358      0.722      -1.397       0.974
Attitude:Reliability                              -2.0281      0.649     -3.127      0.003      -3.327      -0.729
Attitude:Reliability:C(Network)[T.Other]          -1.1068      0.917     -1.207      0.233      -2.943       0.730
Attitude:Reliability:C(Network)[T.Tesla]          -0.2298      0.917     -0.250      0.803      -2.066       1.607
Range:Reliability                                  0.1286      0.649      0.198      0.844      -1.170       1.427
Range:Reliability:C(Network)[T.Other]             -0.1468      0.917     -0.160      0.873      -1.983       1.690
Range:Reliability:C(Network)[T.Tesla]              0.1936      0.917      0.211      0.834      -1.643       2.030
Attitude:Range:Reliability                         1.9973      1.005      1.988      0.052      -0.015       4.009
Attitude:Range:Reliability:C(Network)[T.Other]     1.1234      1.421      0.791      0.432      -1.722       3.969
Attitude:Range:Reliability:C(Network)[T.Tesla]     0.2215      1.421      0.156      0.877   

In [113]:
deep_reload(src)

label_substitutions={
    'C(Network)[T.Other]': 'Non-Tesla Access',
}

out_string=src.analysis.PrintLaTeXTabular(
    model, alpha=.05,
    label_substitutions = label_substitutions
)

print(out_string)

\hline {\small Intercept } & 6.678 & 0.000 \\
\hline {\small Range } & -0.903 & 0.035 \\
\hline {\small Attitude } & 2.633 & 0.000 \\
\hline {\small Attitude:Range } & -2.570 & 0.000 \\
\hline {\small Attitude:Reliability } & -2.028 & 0.003 \\
\hline {\small Attitude:Non-Tesla Access } & 1.406 & 0.021 \\



In [47]:
non_proprietary_sng_us = src.graph.graph_from_json(
    'Outputs/SNG/non_proprietary_sng_us.json'
    )

tesla_sng_us = src.graph.graph_from_json(
    'Outputs/SNG/tesla_sng_us.json'
    )

combined_sng_us = src.graph.graph_from_json(
    'Outputs/SNG/combined_sng_us.json'
    )

In [114]:
deep_reload(src)

nw = [tesla_sng_us, non_proprietary_sng_us, combined_sng_us]

ris = []
rbs = []

for snw in nw:

    x, n, h = src.analysis.redundancy_in_station(snw)

    ris.append(h.mean())

    x, n, h = src.analysis.redundancy_between_stations(
        snw, field = 'time', cutoff = 600
    )

    rbs.append(h.mean())

# x, n, h = src.analysis.redundancy_in_station(non_proprietary_sng_us)
# print(h.mean(), h.median())

# x, n, h = src.analysis.redundancy_between_stations(
#     non_proprietary_sng_us, field = 'time', cutoff = 600)
# print(h.mean())

# x, n, h = src.analysis.redundancy_between_stations(
#     non_proprietary_sng_us, field = 'time', cutoff = 1200)
# print(h.mean())

# x, n, h = src.analysis.redundancy_between_stations(
#     combined_sng_us, field = 'time', cutoff = 600)
# print(h.mean())

In [115]:
risk_attitudes = [.25, .5, .75]
ranges = [200, 300, 400]
reliabilities = [.50, .75, 1]

levels = src.utilities.full_factorial([3, 3, 3, 3])

In [116]:
deep_reload(src)

rta = []
ra = []
rng = []
rel = []
r_is = []
r_bs = []

for idx, file in enumerate(files):
    # print(idx)

    row = levels[idx]

    try:
        costs, values, paths = pkl.load(open(f'Outputs/Experiment/case_{idx}.pkl', 'rb'))
    
        rta_c = src.routing.road_trip_accessibility(
            values,
            # expectation = lambda x: src.routing.super_quantile(
            #     x, risk_attitudes_ranges[row[0]])
        ) / 3600
    
        rta.append(rta_c)
        ra.append(risk_attitudes[row[0]])
        rng.append(ranges[row[1]])
        rel.append(reliabilities[row[2]])
        r_is.append(ris[row[3]])
        r_bs.append(rbs[row[3]])
        
    except:
        print(f'python run_experiment.py -i {idx} &')
        print(idx, sum([type(v) is str for v in values.values()]))

    

In [117]:
rng_n = [(r - 200) / 200 for r in rng]
rel_n = [(r - .5) / .5 for r in rel]
ra_n = [(r - .25) / .5 for r in ra]
r_is_n = [(r - min(ris)) / (max(ris) - min(ris)) for r in r_is]
r_bs_n = [(r - min(rbs)) / (max(rbs) - min(rbs)) for r in r_bs]

In [118]:
data = {
    'Attitude': ra_n,
    'Range': rng_n,
    'Reliability': rel_n,
    'RIS': r_is_n,
    'RBS': r_bs_n,
    'RTA': rta
}

df2 = pd.DataFrame(
    data = data,
)

In [119]:
df_sel = df2

model = smf.ols(
    'RTA ~ Attitude * Range * Reliability * RIS', data = df_sel
).fit()

In [120]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    RTA   R-squared:                       0.910
Model:                            OLS   Adj. R-squared:                  0.889
Method:                 Least Squares   F-statistic:                     43.70
Date:                Wed, 15 May 2024   Prob (F-statistic):           4.44e-28
Time:                        09:20:27   Log-Likelihood:                -23.244
No. Observations:                  81   AIC:                             78.49
Df Residuals:                      65   BIC:                             116.8
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                          6.8002      0.229     29.666      0.000       6.342       7.258
Attitude                           3.6398      0.355     10.249      0.000       2.931       4.349
Range                             -0.9208      0.355     -2.593      0.012      -1.630      -0.212
Attitude:Range                    -3.5590      0.550     -6.469      0.000      -4.658      -2.460
Reliability                        0.0462      0.355      0.130      0.897      -0.663       0.755
Attitude:Reliability              -2.7723      0.550     -5.039      0.000      -3.871      -1.674
Range:Reliability                 -0.0010      0.550     -0.002      0.999      -1.100       1.098
Attitude:Range:Reliability         2.7543      0.852      3.232      0.002       1.052       4.456
RIS                               -0.3266      0.379     -0.862      0.392      -1.083       0.430
Attitude:RIS                      -1.3394      0.587     -2.283      0.026      -2.511      -0.168
Range:RIS                         -0.0174      0.587     -0.030      0.976      -1.189       1.154
Attitude:Range:RIS                 1.2404      0.909      1.365      0.177      -0.575       3.055
Reliability:RIS                   -0.3499      0.587     -0.596      0.553      -1.521       0.822
Attitude:Reliability:RIS           0.6814      0.909      0.750      0.456      -1.134       2.496
Range:Reliability:RIS              0.3311      0.909      0.364      0.717      -1.484       2.146
Attitude:Range:Reliability:RIS    -0.7042      1.408     -0.500      0.619      -3.516       2.108
==============================================================================
Omnibus:                        6.889   Durbin-Watson:                   1.134
Prob(Omnibus):                  0.032   Jarque-Bera (JB):               12.455
Skew:                          -0.018   Prob(JB):                      0.00197
Kurtosis:                       4.921   Cond. No.                         91.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [122]:
deep_reload(src)

label_substitutions={
    'C(Network)[T.Other]': 'Non-Tesla',
    'RIS': 'Redundancy-IS',
}

out_string=src.analysis.PrintLaTeXTabular(
    model, alpha=.05,
    label_substitutions = label_substitutions
)

print(out_string)

\hline {\small Intercept } & 6.800 & 0.000 \\
\hline {\small Range } & -0.921 & 0.012 \\
\hline {\small Attitude } & 3.640 & 0.000 \\
\hline {\small Attitude:Range } & -3.559 & 0.000 \\
\hline {\small Attitude:Reliability } & -2.772 & 0.000 \\
\hline {\small Attitude:Redundancy-IS } & -1.339 & 0.026 \\
\hline {\small Attitude:Range:Reliability } & 2.754 & 0.002 \\

